## Install OpenAI Library

In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tenacity

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


## Import Library

In [8]:
import openai
import json
import pandas as pd
import numpy as np
import pickle
import tiktoken
from dotenv import dotenv_values
from tenacity import retry, wait_random_exponential, stop_after_attempt

## Add API key

In [9]:
config = dotenv_values(".env")

openai.api_key  = config["OPENAI_API_KEY"]

## Data Frame

In [10]:
dataset_path = "./movie_plot.csv"
df = pd.read_csv(dataset_path)

In [11]:
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [12]:
movies = df[df["Origin/Ethnicity"] == "American"].sort_values("Release Year", ascending=False).head(5000)

In [13]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
17376,2017,Phantom Thread,American,Paul Thomas Anderson,Paul Thomas Anderson (director/screenplay); Da...,drama,https://en.wikipedia.org/wiki/Phantom_Thread,"In 1954 London, renowned fashion designer Reyn..."
17243,2017,"Everything, Everything",American,Stella Meghie,Stella Meghie (director); J. Mills Goodloe (sc...,"romance, drama","https://en.wikipedia.org/wiki/Everything,_Ever...","Eighteen-year-old Maddy suffers from SCID, an ..."
17241,2017,Alien: Covenant,American,Ridley Scott,"Ridley Scott (director); Michael Green, Jack P...","sci-fi, horror",https://en.wikipedia.org/wiki/Alien:_Covenant,"In a prologue, business magnate Peter Weyland ..."
17240,2017,Paris Can Wait,American,Eleanor Coppola,Eleanor Coppola (director/screenplay); Diane L...,"comedy, romance",https://en.wikipedia.org/wiki/Paris_Can_Wait,Anne (Diane Lane) is in Cannes with her husban...
17239,2017,The Wall,American,Doug Liman,Doug Liman (director); Dwain Worrell (screenpl...,"drama, thriller",https://en.wikipedia.org/wiki/The_Wall_(2017_f...,"During the Iraq War, U.S. Army Staff Sergeant ..."
...,...,...,...,...,...,...,...,...
12478,1995,Jury Duty,American,John Fortenberry,"Pauly Shore, Tia Carrere, Abe Vigoda",comedy,https://en.wikipedia.org/wiki/Jury_Duty_(film),Tommy Collins is an unemployed erotic dancer l...
12506,1995,Mortal Kombat,American,Paul W. S. Anderson,"Robin Shou, Linden Ashby, Bridgette Wilson, Ch...",fantasy,https://en.wikipedia.org/wiki/Mortal_Kombat_(f...,Mortal Kombat is a fighting tournament between...
12480,1995,A Kid in King Arthur's Court,American,Michael Gottlieb,"Thomas Ian Nicholas, Joss Ackland",comedy,https://en.wikipedia.org/wiki/A_Kid_in_King_Ar...,Calvin Fuller is a nerdy young boy living in t...
12508,1995,Mr. Holland's Opus,American,Stephen Herek,"Richard Dreyfuss, Glenne Headly, Jay Thomas, O...",drama,https://en.wikipedia.org/wiki/Mr._Holland%27s_...,"In Portland, Oregon in 1965, Glenn Holland is ..."


In [14]:
movie_plots = movies["Plot"].values

## Declare Functions

In [15]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):
    
    # replace new lines, which can negatively affect performance.
    text = text.replace("\n", " ")
    
    return openai.Embedding.create(input=text, model=model)["data"][0]["embedding"]

In [16]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embeddings cache
embedding_cache_path = "movie_embedding_demo.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)
    
# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string,
    model="text-embedding-ada-002",
    embedding_cache=embedding_cache
):
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPEN_AI FOR {string[:20]}")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [17]:
embedding_from_string("Fried Chicken is the best!!")

[0.020977254956960678,
 -0.016692114993929863,
 -0.028326768428087234,
 -0.019345413893461227,
 -0.019146105274558067,
 -0.02431567944586277,
 -0.024539900943636894,
 -0.021537812426686287,
 -0.007754359859973192,
 -0.01034537423402071,
 0.011503857560455799,
 0.0261841993778944,
 0.0006489215884357691,
 -0.003911436069756746,
 -0.0069758095778524876,
 0.024527443572878838,
 0.02439041994512081,
 0.013627742417156696,
 0.007181346882134676,
 -0.015683114528656006,
 -0.022185565903782845,
 0.0025100456550717354,
 -0.016629831865429878,
 -0.024813950061798096,
 0.00652736471965909,
 0.01681668311357498,
 -0.015969621017575264,
 -0.035178009420633316,
 0.025486618280410767,
 0.004316281992942095,
 0.03253716975450516,
 0.015932250767946243,
 -0.02295788750052452,
 -0.0077232178300619125,
 -0.013490716926753521,
 0.020902514457702637,
 -0.025411877781152725,
 -0.009355058893561363,
 0.009292774833738804,
 0.014873422682285309,
 0.01973157562315464,
 -0.01175922155380249,
 0.009666479192674

## Tiktoken

In [18]:
enc = tiktoken.encoding_for_model("text-embedding-ada-002")

In [19]:
total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])

In [20]:
total_tokens
cost = total_tokens * (.0004 / 1000)
print(f"Estimated cost ${cost:.2f}")

Estimated cost $1.45


## Insert Your Input

In [23]:
plot_embeddings = [embedding_from_string(plot, model="text-embedding-ada-002") for plot in movie_plots]

In [22]:
len(plot_embeddings)

5000

In [24]:
plot_embeddings